# 🧪 Defect Detection in Manufacturing
This notebook trains a CNN model to classify images as defective or non-defective.

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import pickle


## 📥 Load CSV Metadata

In [ ]:
df = pd.read_csv("data/image_labels.csv")
df["label"] = df["label"].map({"defective": 1, "non-defective": 0})
df.head()


## 🔄 Image Generators for Training/Validation

In [ ]:
IMG_SIZE = (64, 64)
BATCH_SIZE = 4

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)

train_gen = ImageDataGenerator(rescale=1./255)
val_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_dataframe(train_df, directory=".", x_col="image_path", y_col="label",
                                           target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')

val_data = val_gen.flow_from_dataframe(val_df, directory=".", x_col="image_path", y_col="label",
                                       target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')


## 🧠 Build and Train CNN Model

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(*IMG_SIZE, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_data, validation_data=val_data, epochs=5)


## 📊 Evaluate Model

In [ ]:
val_data.reset()
y_true = val_data.classes
y_pred = model.predict(val_data).flatten()
y_pred = np.where(y_pred > 0.5, 1, 0)

print(classification_report(y_true, y_pred))


## 💾 Save Model and Class Indices

In [ ]:
os.makedirs("model", exist_ok=True)
model.save("model/cnn_defect_model.h5")
with open("model/class_map.pkl", "wb") as f:
    pickle.dump(train_data.class_indices, f)
